# Predicting Satisfiability of SAT-3 Problems
## Solving the Classification Task using a ***Graphical Neural Network (GNN)***

A *satisfiability problem* (SAT problem) is a family of problems problem, where given a Boolean expression written using only the logical connectives AND, OR, NOT, variables and parentheses, we examine if there is some truth-assignment to the variables that will make the entire expression true. A **SAT-3** problem is a special case of SAT problem, where Boolean expression should have very strict form : the conjunctive normal form (CNF). A CNF is a conjunction of one or more clauses, where a clause is a disjunction of literals; otherwise put, it is an AND of ORs. In the SAT-3 problem, every disjunction consists of 3 literals. An example of a SAT-3 problem instance is the following:
$$(x_1 \vee x_2 \vee \neg x_4) \wedge (\neg x_3 \vee x_4 \vee x_2)$$

The SAT-3 problem is NP-complete and many solvers have been developed in order to prove the (un)satisfiability of the problems and point a truth assignment of the variables. However, not many large instances can be solved by traditional solvers and, thus, some researchers have come up with the idea to use Machine Learning methods in order to solve those problems, given that with the models we cannot have a guaranteed accuracy, but if the model generalizes well, then we would be able to solve some difficult problem-instances that the state-of-the-art solvers cannot.<br>

In this assignment we will try predicting the satisfiability of the instances using a *Graphical Neural Network (GNN)* and a *Long Short-Term Memory network (LSTM)*. In this notebook will be explained the process of using a GNN.

In [1]:
%load_ext autoreload
%autoreload 2

In [22]:
import random
__counter__ = random.randint(0,2e9)

from IPython.display import HTML, display

In [3]:
import os, shutil
import json

from tuning import tune_parameters
from data_loader import dataset_processing
from train import training, testing

Device is: cuda:0


A helper function to ensure that the dataset is made from scratch when NN is tested. 

In [4]:
def delete_folder_contents(folders):
    for folder in folders:
        for filename in os.listdir(folder):
            file_path = os.path.join(folder, filename)
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)

### Test set sampled from the same distribution as the training and validation sets 

In order to train the model, a *training set* is used alongside with a *validation set*. The latter is used to measure the performance of the model. The training and validation set were extracted from the 80% (60%-20%) of the data and the rest 20% was kept to evaluate the model after its training (*test set*).  

Erase the previous dataset created by PyTorch

In [5]:
delete_folder_contents(["./raw", "./processed"])

**Create the dataset**: The dataset is created from the raw data in such a format that it can be loader by the *DataLoader* module of torch.geometric. <br>

The dataset must be provided to Pytorch in a graph format, so in this preprocessing are constructed the *vertices, the edges, the labels* etc.<br>

The modelling of the classification problem to a graph problem will be explained by providing an example. The SAT-3 problem:
$$(x_1 \vee x_2 \vee \neg x_3) \wedge (\neg x_3 \vee \neg x_1 \vee x_2)$$

is translated to the following graph 

<img src="./plots/graph.jpg" height=250 width=300>

<b>Note:</b> The edges with different color connect different vertices and are considered as such in the *torch.geometric* format.

In [6]:
pos_weight = dataset_processing(separate_test=False)

Start the data processing...

Satisfiable CNFs   : 3701
Unsatisfiable CNFs : 2700

Ratio of SAT   : 0.5782
Ratio of UNSAT : 0.4218

Training set size: 5120
Test set size: 1280

Processing completed.


**Tune parameters**: Tune the parameters of the model (GNN itself) as well as parameters regarding the training process e.g. *batch-size*, *weight decay* etc. More specifically, the parameters that are tuned are the following.
<ol><li>Model parameters:</li>
    <ul> 
        <li>Number of layers</li>
        <li>Dropout rate</li>
        <li>Neurons' density in its last Linear Layers</li>
        <li>Embedding size for the Graph Transformer Operator (TransformerConv)</li>
        <li>Attention heads for the Graph Transformer Operator (TransformerConv)</li><br>
    </ul>
    <li>Training parameters:</li>
    <ul> 
        <li>Batch size used</li>
        <li>Learning rate</li>
        <li>Weight decay</li>
    </ul>
</ol>

The parameters of the final model are the ones that result in the minimum ***validation error***. <br>
Also, ***early stopping*** is used in order to avoid *overfitting*.

In [7]:
# Tune parameters
best_parameters = tune_parameters(pos_weight=pos_weight, model_name='./final_model_same_sets.pth')

# Show best parameters
print(f'Best hyperparameters were: {best_parameters}')
# Store best parameters
with open('./best_parameters_same_sets.txt', 'w') as f:
    f.write(json.dumps(best_parameters))

f.close()


Test number 1 | Start testing new parameter-combination...

Dataset loading...


Processing...
100%|█████████████████████████████████████████████████████████████████████████████| 5120/5120 [00:07<00:00, 689.58it/s]
Done!


Dataset loading completed

Model loading...
Model loading completed

EPOCH | 0
Training Loss   : 0.6465
Validation Loss : 0.5362

EPOCH | 1
Training Loss   : 0.3921
Validation Loss : 1.0453

EPOCH | 2
Training Loss   : 0.3496
Validation Loss : 1.7816

EPOCH | 3
Training Loss   : 0.3438
Validation Loss : 0.6968

EPOCH | 4
Training Loss   : 0.3281
Validation Loss : 0.6758

EPOCH | 5
Training Loss   : 0.3099
Validation Loss : 2.0693

EPOCH | 6
Training Loss   : 0.2985
Validation Loss : 0.6265

EPOCH | 7
Training Loss   : 0.2855
Validation Loss : 1.1371

EPOCH | 8
Training Loss   : 0.2633
Validation Loss : 2.7293

EPOCH | 9
Training Loss   : 0.2675
Validation Loss : 0.2841

EPOCH | 10
Training Loss   : 0.2351
Validation Loss : 0.6169

EPOCH | 11
Training Loss   : 0.2308
Validation Loss : 0.9951

EPOCH | 12
Training Loss   : 0.2290
Validation Loss : 0.2298

EPOCH | 13
Training Loss   : 0.2199
Validation Loss : 0.6966

EPOCH | 14
Training Loss   : 0.2112
Validation Loss : 0.7256

EPOCH | 15


Validation Loss : 1.1085

EPOCH | 5
Training Loss   : 0.3939
Validation Loss : 0.4182

EPOCH | 6
Training Loss   : 0.3816
Validation Loss : 0.9605

EPOCH | 7
Training Loss   : 0.3572
Validation Loss : 0.9000

EPOCH | 8
Training Loss   : 0.3161
Validation Loss : 0.5725

EPOCH | 9
Training Loss   : 0.2961
Validation Loss : 0.2788

EPOCH | 10
Training Loss   : 0.2555
Validation Loss : 0.8720

EPOCH | 11
Training Loss   : 0.2417
Validation Loss : 0.6274

EPOCH | 12
Training Loss   : 0.2299
Validation Loss : 0.1958

EPOCH | 13
Training Loss   : 0.2500
Validation Loss : 0.2182

EPOCH | 14
Training Loss   : 0.2094
Validation Loss : 0.6166

EPOCH | 15
Training Loss   : 0.2274
Validation Loss : 0.2278

EPOCH | 16
Training Loss   : 0.2006
Validation Loss : 1.0026

EPOCH | 17
Training Loss   : 0.1863
Validation Loss : 0.4599

EPOCH | 18
Training Loss   : 0.1789
Validation Loss : 0.1429

EPOCH | 19
Training Loss   : 0.1542
Validation Loss : 0.1150

EPOCH | 20
Training Loss   : 0.1491
Validation Lo

Training Loss   : 0.2815
Validation Loss : 1.8104

EPOCH | 6
Training Loss   : 0.2531
Validation Loss : 0.2548

EPOCH | 7
Training Loss   : 0.2282
Validation Loss : 0.2378

EPOCH | 8
Training Loss   : 0.2077
Validation Loss : 0.4206

EPOCH | 9
Training Loss   : 0.2027
Validation Loss : 2.9819

EPOCH | 10
Training Loss   : 0.2141
Validation Loss : 0.2447

EPOCH | 11
Training Loss   : 0.1937
Validation Loss : 0.4174

EPOCH | 12
Training Loss   : 0.1829
Validation Loss : 0.2464

EPOCH | 13
Training Loss   : 0.1684
Validation Loss : 0.3724

EPOCH | 14
Training Loss   : 0.1659
Validation Loss : 0.3252

EPOCH | 15
Training Loss   : 0.1619
Validation Loss : 0.1678

EPOCH | 16
Training Loss   : 0.1522
Validation Loss : 0.3506

EPOCH | 17
Training Loss   : 0.1420
Validation Loss : 0.1567

EPOCH | 18
Training Loss   : 0.1397
Validation Loss : 0.1811

EPOCH | 19
Training Loss   : 0.1316
Validation Loss : 0.1559

EPOCH | 20
Training Loss   : 0.1266
Validation Loss : 0.1233

EPOCH | 21
Training Los

...

Validation Loss : 0.1100

EPOCH | 5
Training Loss   : 0.1232
Validation Loss : 0.0881

EPOCH | 6
Training Loss   : 0.1064
Validation Loss : 0.0933

EPOCH | 7
Training Loss   : 0.1269
Validation Loss : 0.1070

EPOCH | 8
Training Loss   : 0.1029
Validation Loss : 0.0891

EPOCH | 9
Training Loss   : 0.0970
Validation Loss : 0.0890

EPOCH | 10
Training Loss   : 0.1005
Validation Loss : 0.1037

EPOCH | 11
Training Loss   : 0.0985
Validation Loss : 0.0870

EPOCH | 12
Training Loss   : 0.0918
Validation Loss : 0.0860

EPOCH | 13
Training Loss   : 0.0870
Validation Loss : 0.0817

EPOCH | 14
Training Loss   : 0.0854
Validation Loss : 0.0808

EPOCH | 15
Training Loss   : 0.0824
Validation Loss : 0.0760

EPOCH | 16
Training Loss   : 0.0806
Validation Loss : 0.0721

EPOCH | 17
Training Loss   : 0.0777
Validation Loss : 0.0750

EPOCH | 18
Training Loss   : 0.0748
Validation Loss : 0.0704

EPOCH | 19
Training Loss   : 0.0638
Validation Loss : 0.0630

EPOCH | 20
Training Loss   : 0.0615
Validation Lo

Training Loss   : 0.2039
Validation Loss : 0.1754

EPOCH | 8
Training Loss   : 0.1775
Validation Loss : 0.1926

EPOCH | 9
Training Loss   : 0.1523
Validation Loss : 0.1496

EPOCH | 10
Training Loss   : 0.1331
Validation Loss : 0.1317

EPOCH | 11
Training Loss   : 0.1178
Validation Loss : 0.1486

EPOCH | 12
Training Loss   : 0.1103
Validation Loss : 0.1222

EPOCH | 13
Training Loss   : 0.1270
Validation Loss : 0.1006

EPOCH | 14
Training Loss   : 0.1041
Validation Loss : 0.0905

EPOCH | 15
Training Loss   : 0.1001
Validation Loss : 0.0814

EPOCH | 16
Training Loss   : 0.0941
Validation Loss : 0.0750

EPOCH | 17
Early stopping activated, with training and validation loss difference: 0.0006

Test number 39 | Start testing new parameter-combination...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

EPOCH | 0
Training Loss   : 16.4411
Validation Loss : 0.5980

EPOCH | 1
Training Loss   : 0.5875
Validation Loss : 0.5871

EPOCH | 2
Training Loss   : 0.

Training Loss   : 0.1041
Validation Loss : 0.0844

EPOCH | 7
Training Loss   : 0.1002
Validation Loss : 0.0844

EPOCH | 8
Training Loss   : 0.0852
Validation Loss : 0.1913

EPOCH | 9
Training Loss   : 0.0887
Validation Loss : 0.0985

EPOCH | 10
Training Loss   : 0.0630
Validation Loss : 0.0906

EPOCH | 11
Training Loss   : 0.0590
Validation Loss : 0.0597

EPOCH | 12
Training Loss   : 0.0568
Validation Loss : 0.0520

EPOCH | 13
Training Loss   : 0.0460
Validation Loss : 0.0393

EPOCH | 14
Training Loss   : 0.0403
Validation Loss : 0.0364

EPOCH | 15
Training Loss   : 0.0404
Validation Loss : 0.2565

EPOCH | 16
Training Loss   : 0.0385
Validation Loss : 0.0614

EPOCH | 17
Training Loss   : 0.0335
Validation Loss : 0.0757

EPOCH | 18
Training Loss   : 0.0247
Validation Loss : 0.0226

EPOCH | 19
Training Loss   : 0.0206
Validation Loss : 0.0222

EPOCH | 20
Training Loss   : 0.0178
Validation Loss : 0.0930

EPOCH | 21
Training Loss   : 0.0197
Validation Loss : 0.0198

EPOCH | 22
Training Lo

Dataset loading completed

Model loading...
Model loading completed

EPOCH | 0
Training Loss   : 0.7342
Validation Loss : 0.5743

EPOCH | 1
Training Loss   : 0.5497
Validation Loss : 0.4858

EPOCH | 2
Training Loss   : 0.3594
Validation Loss : 0.5131

EPOCH | 3
Training Loss   : 0.2792
Validation Loss : 1.2774

EPOCH | 4
Training Loss   : 0.2414
Validation Loss : 1.9130

EPOCH | 5
Training Loss   : 0.1839
Validation Loss : 2.1070

EPOCH | 6
Training Loss   : 0.1366
Validation Loss : 1.7802

EPOCH | 7
Training Loss   : 0.1245
Validation Loss : 0.6295

EPOCH | 8
Training Loss   : 0.1219
Validation Loss : 1.0231

EPOCH | 9
Training Loss   : 0.1256
Validation Loss : 0.7379

EPOCH | 10
Training Loss   : 0.1075
Validation Loss : 0.1056

EPOCH | 11
Training Loss   : 0.0995
Validation Loss : 0.8356

EPOCH | 12
Training Loss   : 0.0914
Validation Loss : 0.3498

EPOCH | 13
Training Loss   : 0.0835
Validation Loss : 0.2193

EPOCH | 14
Training Loss   : 0.0833
Validation Loss : 0.2962

EPOCH | 15


Training Loss   : 0.5861
Validation Loss : 0.5853

EPOCH | 23
Training Loss   : 0.5861
Validation Loss : 0.5853

EPOCH | 24
Training Loss   : 0.5861
Validation Loss : 0.5853

EPOCH | 25
Training Loss   : 0.5861
Validation Loss : 0.5853

EPOCH | 26
Training Loss   : 0.5861
Validation Loss : 0.5853

EPOCH | 27
Training Loss   : 0.5860
Validation Loss : 0.5853

EPOCH | 28
Training Loss   : 0.5860
Validation Loss : 0.5853

EPOCH | 29
Training Loss   : 0.5860
Validation Loss : 0.5853

EPOCH | 30
Training Loss   : 0.5860
Validation Loss : 0.5853

EPOCH | 31
Training Loss   : 0.5860
Validation Loss : 0.5853

EPOCH | 32
Training Loss   : 0.5860
Validation Loss : 0.5853

EPOCH | 33
Training Loss   : 0.5860
Validation Loss : 0.5853

EPOCH | 34
Training Loss   : 0.5860
Validation Loss : 0.5853

EPOCH | 35
Training Loss   : 0.5859
Validation Loss : 0.5853

EPOCH | 36
Training Loss   : 0.5859
Validation Loss : 0.5853

EPOCH | 37
Training Loss   : 0.5859
Validation Loss : 0.5853

EPOCH | 38
Training

**Train with the best parameters**: The GNN is trained using the selected parameters.

In [8]:
# Access the best parameters in order to train final model
with open('./best_parameters_same_sets.txt') as f:
    data = f.read()

best_parameters_loaded = json.loads(data)

print('\nNow training with the best parameters\n')
training(params=best_parameters_loaded, model_name='./final_model_same_sets.pth', make_err_logs=True)


Now training with the best parameters

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

EPOCH | 0
Training Loss   : 0.3590
Validation Loss : 0.2296

EPOCH | 1
Training Loss   : 0.1939
Validation Loss : 0.1657

EPOCH | 2
Training Loss   : 0.1717
Validation Loss : 0.1703

EPOCH | 3
Training Loss   : 0.1559
Validation Loss : 0.1363

EPOCH | 4
Training Loss   : 0.1423
Validation Loss : 0.1142

EPOCH | 5
Training Loss   : 0.1423
Validation Loss : 0.1114

EPOCH | 6
Training Loss   : 0.1246
Validation Loss : 0.1415

EPOCH | 7
Training Loss   : 0.1313
Validation Loss : 0.1090

EPOCH | 8
Training Loss   : 0.1264
Validation Loss : 0.1077

EPOCH | 9
Training Loss   : 0.1103
Validation Loss : 0.0952

EPOCH | 10
Training Loss   : 0.1120
Validation Loss : 0.0858

EPOCH | 11
Training Loss   : 0.0973
Validation Loss : 0.0953

EPOCH | 12
Training Loss   : 0.0988
Validation Loss : 0.0791

EPOCH | 13
Training Loss   : 0.0867
Validation Loss : 0.0977

EPOCH | 14
Tra

0.17032846371876076

The following Figure shows the ***training and validation set errors***, the epoch where the ***early stopping*** was activated, as well as the epoch of the final ***selected model***.

In [9]:
display(HTML('<img src="plots/train_valid_error.png?%d" height=400 width=400>' % __counter__))

**Test the model**: Predict the satisfiability of the clauses in the *Test Set* and get the corresponding metrics.

In [10]:
print('\nResults on the test set:\n')
testing(params=best_parameters_loaded, model_name='./final_model_same_sets.pth',)


Results on the test set:

Dataset loading...


Processing...
100%|█████████████████████████████████████████████████████████████████████████████| 1280/1280 [00:01<00:00, 656.14it/s]
Done!


Dataset loading completed

Model loading...
Model loading completed


Test set metrics:

 Confusion matrix: 
 [[510 134]
 [  0 636]]
F1 Score  : 0.9047
Accuracy  : 0.8953
Precision : 1.0000
Recall    : 0.8260
ROC AUC   : 0.9130
Test Loss : 0.1717989294193103


The following **Figures** show:
<ol>
<li>The <b>confusion matrix</b> for the test set-prediction</li>
<li>The <b>ROC-AUC curve</b> for the test set-prediction</li>
<li>The <b>precision recall curve</b> for the test set-prediction</li>
</ol>

In [11]:
print("\n1.")
display(HTML('<img src="plots/cm.png?%d" height=500 width=500>' % __counter__))
print("2.")
display(HTML('<img src="plots/roc_auc.png?%d" height=450 width=450>' % __counter__))
print("3.")
display(HTML('<img src="plots/pr.png?%d" height=450 width=450>' % __counter__))


1.


2.


3.


### Test set sampled from a different distribution from the training and validation sets 

Here, again, in order to train the model, a training set is used alongside with a validation set. However, now, they are both sampled randomly from the smaller 3-SAT problems i.e. have less than 250 variables. The test set consists of 200 problem instances that have 250 variables and 1065 clauses each. Since in this case we would like to predict the satisfiability of the problems that the well known algorithms cannot predict, ***we want to measure the performance of the model for bigger instances that it has not seen during its training***.

The training-tuning process is the same as showcased above.

In [12]:
# Erase the previous dataset created by PyTorch
delete_folder_contents(["./raw", "./processed"])

In [13]:
# Create the dataset
pos_weight = dataset_processing(separate_test=True)

Start the data processing...

Satisfiable CNFs   : 3601
Unsatisfiable CNFs : 2600

Ratio of SAT   : 0.5807
Ratio of UNSAT : 0.4193

Training set size: 6200
Test set size: 200

Processing completed.


In [14]:
# Tune parameters
best_parameters = tune_parameters(pos_weight=pos_weight, model_name='./final_model_diff_sets.pth')

# Show best parameters
print(f'Best hyperparameters were: {best_parameters}')
# Store best parameters
with open('best_parameters_diff_test.txt', 'w') as f:
    f.write(json.dumps(best_parameters))

f.close()


Test number 1 | Start testing new parameter-combination...

Dataset loading...


Processing...
100%|█████████████████████████████████████████████████████████████████████████████| 6200/6200 [00:08<00:00, 721.87it/s]
Done!


Dataset loading completed

Model loading...
Model loading completed

EPOCH | 0
Training Loss   : 0.5296
Validation Loss : 2.8038

EPOCH | 1
Training Loss   : 0.4610
Validation Loss : 2.6809

EPOCH | 2
Training Loss   : 0.3170
Validation Loss : 0.5328

EPOCH | 3
Training Loss   : 0.2176
Validation Loss : 0.2368

EPOCH | 4
Training Loss   : 0.2138
Validation Loss : 5.4041

EPOCH | 5
Training Loss   : 0.1942
Validation Loss : 0.5270

EPOCH | 6
Training Loss   : 0.1824
Validation Loss : 0.3419

EPOCH | 7
Training Loss   : 0.1633
Validation Loss : 0.2681

EPOCH | 8
Training Loss   : 0.1450
Validation Loss : 0.2935

EPOCH | 9
Training Loss   : 0.1389
Validation Loss : 0.3008

EPOCH | 10
Training Loss   : 0.1384
Validation Loss : 0.1274

EPOCH | 11
Training Loss   : 0.1322
Validation Loss : 0.1192

EPOCH | 12
Training Loss   : 0.1293
Validation Loss : 0.7558

EPOCH | 13
Training Loss   : 0.1484
Validation Loss : 0.7029

EPOCH | 14
Training Loss   : 0.1511
Validation Loss : 0.8073

EPOCH | 15


Training Loss   : 0.4181
Validation Loss : 0.4048

EPOCH | 4
Training Loss   : 0.2713
Validation Loss : 0.6580

EPOCH | 5
Training Loss   : 0.1972
Validation Loss : 0.1588

EPOCH | 6
Training Loss   : 0.1611
Validation Loss : 0.1082

EPOCH | 7
Training Loss   : 0.1344
Validation Loss : 0.6483

EPOCH | 8
Training Loss   : 0.1156
Validation Loss : 0.3582

EPOCH | 9
Training Loss   : 0.0955
Validation Loss : 2.1791

EPOCH | 10
Training Loss   : 0.1024
Validation Loss : 0.1253

EPOCH | 11
Training Loss   : 0.0926
Validation Loss : 0.2613

EPOCH | 12
Training Loss   : 0.0683
Validation Loss : 0.0710

EPOCH | 13
Training Loss   : 0.0658
Validation Loss : 0.1460

EPOCH | 14
Training Loss   : 0.0693
Validation Loss : 0.1044

EPOCH | 15
Training Loss   : 0.1010
Validation Loss : 0.0769

EPOCH | 16
Training Loss   : 0.0457
Validation Loss : 0.0372

EPOCH | 17
Training Loss   : 0.0356
Validation Loss : 0.1179

EPOCH | 18
Training Loss   : 0.0249
Validation Loss : 0.0190

EPOCH | 19
Training Loss 

Validation Loss : 0.5811

EPOCH | 36
Training Loss   : 0.5813
Validation Loss : 0.5811

EPOCH | 37
Training Loss   : 0.5813
Validation Loss : 0.5811

EPOCH | 38
Training Loss   : 0.5813
Validation Loss : 0.5811

EPOCH | 39
Training Loss   : 0.5813
Validation Loss : 0.5811

EPOCH | 40
Training Loss   : 0.5813
Validation Loss : 0.5811

EPOCH | 41
Training Loss   : 0.5813
Validation Loss : 0.5811

EPOCH | 42
Training Loss   : 0.5813
Validation Loss : 0.5811

EPOCH | 43
Training Loss   : 0.5813
Validation Loss : 0.5811

EPOCH | 44
Training Loss   : 0.5813
Validation Loss : 0.5811

EPOCH | 45
Training Loss   : 0.5813
Validation Loss : 0.5811

EPOCH | 46
Training Loss   : 0.5813
Validation Loss : 0.5811

EPOCH | 47
Training Loss   : 0.5813
Validation Loss : 0.5811

EPOCH | 48
Training Loss   : 0.5813
Validation Loss : 0.5811

EPOCH | 49
Training Loss   : 0.5813
Validation Loss : 0.5811

EPOCH | 50
Training Loss   : 0.5813
Validation Loss : 0.5811

Finishing training with best training loss: 

...

Validation Loss : 0.5571

EPOCH | 30
Training Loss   : 0.0437
Validation Loss : 1.2651

EPOCH | 31
Training Loss   : 0.0369
Validation Loss : 0.9572

EPOCH | 32
Early stopping activated, with training and validation loss difference: 0.0110

Test number 31 | Start testing new parameter-combination...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

EPOCH | 0
Training Loss   : 0.8551
Validation Loss : 0.6596

EPOCH | 1
Training Loss   : 0.5729
Validation Loss : 0.8083

EPOCH | 2
Training Loss   : 0.5635
Validation Loss : 0.8474

EPOCH | 3
Training Loss   : 0.5554
Validation Loss : 1.1323

EPOCH | 4
Training Loss   : 0.5471
Validation Loss : 1.3277

EPOCH | 5
Training Loss   : 0.5496
Validation Loss : 1.7252

EPOCH | 6
Training Loss   : 0.5336
Validation Loss : 2.5070

EPOCH | 7
Training Loss   : 0.5247
Validation Loss : 3.1891

EPOCH | 8
Training Loss   : 0.5248
Validation Loss : 3.2069

EPOCH | 9
Training Loss   : 0.5239
Validation Loss : 2.6364



Validation Loss : 0.0100

EPOCH | 16
Training Loss   : 0.0096
Validation Loss : 0.0053

EPOCH | 17
Training Loss   : 0.0094
Validation Loss : 0.0121

EPOCH | 18
Training Loss   : 0.0070
Validation Loss : 0.0018

EPOCH | 19
Training Loss   : 0.0049
Validation Loss : 0.0024

EPOCH | 20
Training Loss   : 0.0040
Validation Loss : 0.0014

EPOCH | 21
Training Loss   : 0.0034
Validation Loss : 0.0008

EPOCH | 22
Training Loss   : 0.0031
Validation Loss : 0.0007

EPOCH | 23
Training Loss   : 0.0029
Validation Loss : 0.0007

EPOCH | 24
Training Loss   : 0.0028
Validation Loss : 0.0008

EPOCH | 25
Training Loss   : 0.0027
Validation Loss : 0.0008

EPOCH | 26
Training Loss   : 0.0027
Validation Loss : 0.0014

EPOCH | 27
Training Loss   : 0.0025
Validation Loss : 0.0011

EPOCH | 28
Training Loss   : 0.0022
Validation Loss : 0.0013

EPOCH | 29
Training Loss   : 0.0020
Validation Loss : 0.0013

EPOCH | 30
Training Loss   : 0.0019
Validation Loss : 0.0014

EPOCH | 31
Training Loss   : 0.0018
Validati

Dataset loading completed

Model loading...
Model loading completed

EPOCH | 0
Training Loss   : 14.4223
Validation Loss : 0.5761

EPOCH | 1
Training Loss   : 0.5822
Validation Loss : 0.5661

EPOCH | 2
Training Loss   : 0.5822
Validation Loss : 0.5626

EPOCH | 3
Training Loss   : 0.5822
Validation Loss : 0.5616

EPOCH | 4
Training Loss   : 0.5821
Validation Loss : 0.5662

EPOCH | 5
Training Loss   : 0.5821
Validation Loss : 0.5707

EPOCH | 6
Training Loss   : 0.5820
Validation Loss : 0.5759

EPOCH | 7
Training Loss   : 0.5820
Validation Loss : 0.5913

EPOCH | 8
Training Loss   : 0.5819
Validation Loss : 0.6061

EPOCH | 9
Training Loss   : 0.5819
Validation Loss : 0.6130

EPOCH | 10
Training Loss   : 0.5819
Validation Loss : 0.6281

EPOCH | 11
Training Loss   : 0.5819
Validation Loss : 0.6419

EPOCH | 12
Training Loss   : 0.5818
Validation Loss : 0.6548

EPOCH | 13
Training Loss   : 0.5818
Validation Loss : 0.6634

EPOCH | 14
Training Loss   : 0.5818
Validation Loss : 0.7011

EPOCH | 15

Validation Loss : 0.0651

EPOCH | 25
Training Loss   : 0.0041
Validation Loss : 0.0478

EPOCH | 26
Early stopping activated, with training and validation loss difference: 0.0001

Test number 44 | Start testing new parameter-combination...

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

EPOCH | 0
Training Loss   : 0.5606
Validation Loss : 0.6149

EPOCH | 1
Training Loss   : 0.4559
Validation Loss : 0.3298

EPOCH | 2
Training Loss   : 0.2573
Validation Loss : 0.2060

EPOCH | 3
Training Loss   : 0.1615
Validation Loss : 0.1815

EPOCH | 4
Training Loss   : 0.1257
Validation Loss : 0.3385

EPOCH | 5
Training Loss   : 0.0994
Validation Loss : 0.1016

EPOCH | 6
Training Loss   : 0.0857
Validation Loss : 0.4481

EPOCH | 7
Training Loss   : 0.0713
Validation Loss : 0.4248

EPOCH | 8
Training Loss   : 0.0622
Validation Loss : 0.2112

EPOCH | 9
Training Loss   : 0.0632
Validation Loss : 0.0790

EPOCH | 10
Training Loss   : 0.0514
Validation Loss : 0.3649



Dataset loading completed

Model loading...
Model loading completed

EPOCH | 0
Training Loss   : 26.7804
Validation Loss : 0.5915

EPOCH | 1
Training Loss   : 0.5875
Validation Loss : 0.5828

EPOCH | 2
Training Loss   : 0.6265
Validation Loss : 0.5817

EPOCH | 3
Training Loss   : 0.5820
Validation Loss : 0.5820

EPOCH | 4
Training Loss   : 0.5820
Validation Loss : 0.5819

EPOCH | 5
Training Loss   : 0.5820
Validation Loss : 0.5818

EPOCH | 6
Training Loss   : 0.5820
Validation Loss : 0.5817

EPOCH | 7
Training Loss   : 0.5819
Validation Loss : 0.5815

EPOCH | 8
Training Loss   : 0.5819
Validation Loss : 0.5814

EPOCH | 9
Training Loss   : 0.5819
Validation Loss : 0.5813

EPOCH | 10
Training Loss   : 0.5818
Validation Loss : 0.5812

EPOCH | 11
Training Loss   : 0.5818
Validation Loss : 0.5812

EPOCH | 12
Training Loss   : 0.5818
Validation Loss : 0.5811

EPOCH | 13
Training Loss   : 0.5818
Validation Loss : 0.5811

EPOCH | 14
Training Loss   : 0.5818
Validation Loss : 0.5811

EPOCH | 15

In [19]:
# Train with the best parameters

# Access the best parameters in order to train final model
with open('best_parameters_diff_test.txt') as f:
    data = f.read()

best_parameters_loaded = json.loads(data)

print('\nNow training with the best parameters\n')
training(params=best_parameters_loaded, model_name='./final_model_diff_sets.pth', make_err_logs=True)


Now training with the best parameters

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed

EPOCH | 0
Training Loss   : 0.5640
Validation Loss : 0.4948

EPOCH | 1
Training Loss   : 0.4956
Validation Loss : 0.5558

EPOCH | 2
Training Loss   : 0.3086
Validation Loss : 0.5998

EPOCH | 3
Training Loss   : 0.1901
Validation Loss : 0.2172

EPOCH | 4
Training Loss   : 0.1690
Validation Loss : 0.9522

EPOCH | 5
Training Loss   : 0.1383
Validation Loss : 0.1606

EPOCH | 6
Training Loss   : 0.1176
Validation Loss : 0.0810

EPOCH | 7
Training Loss   : 0.1050
Validation Loss : 0.0574

EPOCH | 8
Training Loss   : 0.0828
Validation Loss : 0.0344

EPOCH | 9
Training Loss   : 0.0754
Validation Loss : 0.0360

EPOCH | 10
Training Loss   : 0.0664
Validation Loss : 0.0417

EPOCH | 11
Training Loss   : 0.0552
Validation Loss : 0.0198

EPOCH | 12
Training Loss   : 0.0499
Validation Loss : 0.0207

EPOCH | 13
Training Loss   : 0.0420
Validation Loss : 0.0118

EPOCH | 14
Tra

0.0031733228936037705

In [20]:
# Test the model
print('\nResults on the test set:\n')
testing(params=best_parameters_loaded, model_name='./final_model_diff_sets.pth')


Results on the test set:

Dataset loading...
Dataset loading completed

Model loading...
Model loading completed


Test set metrics:

 Confusion matrix: 
 [[ 33 100]
 [ 67   0]]
F1 Score  : 0.0000
Accuracy  : 0.1650
Precision : 0.0000
Recall    : 0.0000
ROC AUC   : 0.1650
Test Loss : 3.052071818938622


The following **Figures** show:
<ol>
<li>The <b>confusion matrix</b> for the test set-prediction</li>
<li>The <b>ROC-AUC curve</b> for the test set-prediction</li>
<li>The <b>precision recall curve</b> for the test set-prediction</li>
</ol>

In [23]:
print("\n1.")
display(HTML('<img src="plots/cm.png?%d" height=500 width=500>' % __counter__))
print("2.")
display(HTML('<img src="plots/roc_auc.png?%d" height=450 width=450>' % __counter__))
print("3.")
display(HTML('<img src="plots/pr.png?%d" height=450 width=450>' % __counter__))


1.


2.


3.
